## This demo app shows:

[source](https://github.com/facebookresearch/llama-recipes/tree/main)

* How to run Llama2 locally on a Mac using llama-cpp-python and the llama-cpp's quantized Llama2 model
* How to use LangChain to ask Llama general questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

We start by installing necessary requirements and import packages we will be using in this example.
- [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) a simple Python bindings for [llama.cpp](https://github.com/ggerganov/llama.cpp) library
- pypdf gives us the ability to work with pdfs
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

In [1]:
# install all the required packages for the demo
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
# !pip install pypdf sentence-transformers chromadb langchain

In [2]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [3]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model. 

Replace `<path-to-llama-gguf-file>` with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing), 

or to the `ggml-model-q4_0.gguf` file built with the following commands:

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0

```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [5]:

llm = LlamaCpp(
    model_path="./models/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager, 
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       

With the model set up, you are now ready to ask some questions. 

Here is an example of the simplest way to ask the model some general questions.

In [7]:
question = "who wrote the book Innovator's dilemma?Give me just the name"
answer = llm.invoke(question)
answer

Llama.generate: prefix-match hit


 of the author.
Answer: The author of "Innovator's Dilemma" is Clayton Christensen.


llama_print_timings:        load time =    5945.47 ms
llama_print_timings:      sample time =       3.05 ms /    28 runs   (    0.11 ms per token,  9174.31 tokens per second)
llama_print_timings: prompt eval time =    2097.40 ms /     7 tokens (  299.63 ms per token,     3.34 tokens per second)
llama_print_timings:        eval time =    2980.86 ms /    27 runs   (  110.40 ms per token,     9.06 tokens per second)
llama_print_timings:       total time =    3799.73 ms /    34 tokens


' of the author.\nAnswer: The author of "Innovator\'s Dilemma" is Clayton Christensen.'

Alternatively, you can use LangChain's `PromptTemplate` for some flexibility in your prompts and questions.

For more information on LangChain's prompt template visit this [link](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

In [8]:
# a more flexible way to ask Llama general questions using LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    "who wrote {book}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke("innovator's dilemma")

Llama.generate: prefix-match hit




The Innovator's Dilemma is a book written by Clayton Christensen, a Harvard Business School professor and leading authority on innovation and business strategy. The book was first published in 1997 and has since become a classic in the field of entrepreneurship and business management.


llama_print_timings:        load time =    5945.47 ms
llama_print_timings:      sample time =       7.88 ms /    67 runs   (    0.12 ms per token,  8505.78 tokens per second)
llama_print_timings: prompt eval time =    2333.37 ms /     8 tokens (  291.67 ms per token,     3.43 tokens per second)
llama_print_timings:        eval time =   10614.42 ms /    67 runs   (  158.42 ms per token,     6.31 tokens per second)
llama_print_timings:       total time =   13134.44 ms /    75 tokens


Now, let's see how Llama2 hallucinates, because it did not have knowledge about Llama2 at the time it was trained. 
By default it behaves like a know-it-all expert who will not say "I don't know".

In [9]:
prompt = PromptTemplate.from_template(
    "What is {what}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke("llama2")

Llama.generate: prefix-match hit




LLAMA2 (Linguistic and Lexical Analysis MAchine) is a computational model of language processing that is designed to simulate the way humans process language. It is an extension of the original LLAMA model, which was developed in the 1970s and 1980s.
LLAMA2 is a cognitive architecture for language processing that includes a range of components, such as:
1. Lexical processing: This component is responsible for handling the lexicon, or vocabulary, of a language. It includes processes such as word recognition, semantic reasoning, and synonymy resolution.
2. Syntactic processing: This component is responsible for handling the grammatical structure of a sentence. It includes processes such as phrase structure analysis, constituent parsing, and syntactic ambiguity resolution.
3. Semantic processing: This component is responsible for understanding the meaning of a sentence. It includes processes such as semantic role labeling, semantic inference, and pragmatic reasoning.
4. Discourse proces


llama_print_timings:        load time =    5945.47 ms
llama_print_timings:      sample time =      30.19 ms /   256 runs   (    0.12 ms per token,  8478.79 tokens per second)
llama_print_timings: prompt eval time =    1234.41 ms /     6 tokens (  205.73 ms per token,     4.86 tokens per second)
llama_print_timings:        eval time =   33187.15 ms /   255 runs   (  130.15 ms per token,     7.68 tokens per second)
llama_print_timings:       total time =   35060.93 ms /   261 tokens


One way we can fix the hallucinations is to use RAG, to augment it with more recent or custom data that holds the information for it to answer correctly.

First we load the Llama2 paper using LangChain's [PDF loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [11]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./assets-resources/llama2_paper.pdf")
docs = loader.load()
docs[0]

Document(page_content='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodr

In [12]:
# quick check on the loaded document for the correct pages etc
print(len(docs), docs[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


Next we will store our documents. 
There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the `HuggingFaceEmbeddings` and `RecursiveCharacterTextSplitter` to assist in storing the documents.

In [13]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 


In [14]:
# split the loaded documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

In [16]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to the model to answer the question.

It takes close to 2 minutes to return the result (but using other vector stores other than Chroma such as FAISS can take longer) because Llama2 is running on a local Mac. 
To get much faster results, you can use a cloud service with GPU used for inference - see HelloLlamaCloud for a demo.

In [ ]:
question = "What is llama2?"
result = qa_chain.invoke({"query": question})
result